---
<center><h1>Basic intro into pandas</h1></center> 
---
---

<center><h2>Working with pandas DataFrames: grouping</h2></center>
---

## Table of Contents

- [Work with pandas DataFrames: grouping](#Work-with-pandas-DataFrames:-grouping)
    * [Splitting of a DataFrame into groups](#Splitting-of-a-DataFrame-into-groups)
    * [Selection and filtering](#Selection-and-filtering)
    * [Aggregation and function application](#Aggregation-and-function-application)
    - [*Exercise 1*](#Exercise-1)

In [131]:
import pandas as pd
import numpy as np
import random

## Work with pandas DataFrames: grouping

[[back to top]](#Table-of-Contents)

We will a werather station dataset so that we can see what is going on when we group.

In [132]:
from arcgis.features import GeoAccessor, GeoSeriesAccessor
#Import a COVID data layer.  This layer contains the updated stats for each county in the United States
from arcgis.features import FeatureLayer
mylayer = FeatureLayer(("https://services9.arcgis.com/RHVPKKiFTONKtxq3/arcgis/rest/services/NOAA_METAR_current_wind_speed_direction_v1/FeatureServer/0"))
sdf2 = pd.DataFrame.spatial.from_layer(mylayer)
sdf2.head(5)

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,FLT_CATEGORY,SHAPE
0,4992,AGGH,2025-04-16 15:00:00,<NA>,<NA>,6.0,75.2,75.2,100,180,6,0,<NA>,9700,1007.8,"Few Clouds at 400 meters AGL, Few Clouds at 43...",Light Rain,<NA>,<NA>,-9.43,160.047,VFR,"{""x"": 17816350.542999998, ""y"": -1054514.398899..."
1,5233,AGGM,2024-07-05 06:00:00,<NA>,<NA>,6.0,84.2,73.4,70,140,13,0,<NA>,9700,1006.8,"Few Clouds at 580 meters AGL, Scattered Clouds...",No significant weather present at this time.,<NA>,90.9,-8.327,157.263,VFR,"{""x"": 17506437.0806, ""y"": -930237.9125000015, ..."
2,5915,AGGR,2023-11-07 23:00:00,<NA>,<NA>,11.0,86.0,80.6,84,90,6,0,<NA>,9700,1008.8,"Scattered Clouds at 550 meters AGL, Broken Clo...",No significant weather present at this time.,<NA>,101.9,-11.55,160.063,VFR,"{""x"": 17818131.654799998, ""y"": -1294537.687800..."
3,5517,ANYN,2025-04-16 17:00:00,<NA>,<NA>,7.0,80.6,78.8,94,40,11,0,<NA>,9700,1007.8,"Few Clouds at 610 meters AGL, Scattered Clouds...",No significant weather present at this time.,<NA>,89.2,-0.547,166.919,VFR,"{""x"": 18581338.0837, ""y"": -60892.6865000017, ""..."
4,5224,AYMH,2025-04-16 08:00:00,<NA>,<NA>,1634.0,62.6,62.6,100,230,13,0,<NA>,9700,1019.0,"Scattered Clouds, Broken Clouds at 910 meters AGL",Thunderstorms with Rain,<NA>,<NA>,-5.831,144.3,MVFR,"{""x"": 16063402.521499999, ""y"": -650227.3401000..."


In [133]:
sdfUSA = sdf2.loc[sdf2["COUNTRY"].apply(lambda x: "United States Of America" in str(x))]
sdfUSA["State"] = sdfUSA['COUNTRY'].apply(lambda x: str(x).split(",")[0])
sdfUSA["COUNTRY"] = sdfUSA['COUNTRY'].apply(lambda x: str(x).split(",")[1])
sdfUSA

[133]:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
[133]:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,FLT_CATEGORY,SHAPE,State
5950,6060,PAEC,2024-01-16 16:51:00,Chulitna,United States Of America,389.0,30.2,32.0,108,40,19,0,20.7,11000,1013.2,Overcast Cloud Deck at 1300 meters AGL,No significant weather present at this time.,<NA>,<NA>,62.886,-149.583,VFR,"{""x"": -16651503.3913, ""y"": 9072352.268600002, ...",Alaska
5951,3767,PAHP,2022-09-17 01:32:00,Hooper Bay,United States Of America,8.0,51.8,46.4,82,180,81,106,<NA>,3200,979.0,Overcast Cloud Deck at 730 meters AGL,Light Rain,* Maintenance REQUIRED - readings may be compr...,<NA>,61.52,-166.15,IFR,"{""x"": -18495733.3953, ""y"": 8746212.867399998, ...",Alaska
5952,5937,PALR,2023-06-27 00:50:00,Chandalar Lake,United States Of America,579.0,59.0,50.0,72,70,13,0,<NA>,48000,1008.1,"Broken Clouds at 1800 meters AGL, Overcast Clo...",No significant weather present at this time.,<NA>,<NA>,67.5,-148.48,VFR,"{""x"": -16528717.993, ""y"": 10299995.503399998, ...",Alaska
5953,5725,PAOH,2024-05-29 07:56:00,Hoonah,United States Of America,<NA>,45.0,42.1,90,0,0,0,63.7,16000,1015.6,"Broken Clouds at 1500 meters AGL, Overcast Clo...",Automated observation with no human augmentati...,<NA>,<NA>,58.108,-135.448,VFR,"{""x"": -15078002.389, ""y"": 7990039.264200002, ""...",Alaska
5954,4180,PAWI,2024-04-29 23:53:00,Wainwright,United States Of America,12.0,12.9,10.0,88,90,7,0,5.1,16000,1027.1,Scattered Clouds at 60 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,<NA>,70.637,-160.013,VFR,"{""x"": -17812565.6803, ""y"": 11279281.076499999,...",Alaska
6065,4922,KCQT,2024-05-20 14:47:00,Los Angeles,United States Of America,51.0,60.1,52.0,75,0,0,0,<NA>,16000,1016.3,Overcast Cloud Deck at 760 meters AGL,Automated observation with no human augmentati...,3-Hour Pressure Increasing: 0.1 mb,<NA>,34.024,-118.291,MVFR,"{""x"": -13168093.885400001, ""y"": 4032025.095200...",California
6066,517,KL52,2023-04-09 16:55:00,Oceano Cnty,United States Of America,3.0,55.4,55.4,100,250,9,0,<NA>,13000,1020.3,<NA>,Automated observation with no human augmentati...,<NA>,<NA>,35.1,-120.62,<NA>,"{""x"": -13427356.9795, ""y"": 4177479.0622000024,...",California
6083,6002,KQBL,2024-06-14 22:55:00,Hurlbert,United States Of America,9999.0,86.0,75.7,72,<NA>,6,0,<NA>,16000,1012.5,Few Clouds at 2700 meters AGL,Automated observation with no human augmentati...,<NA>,96.0,30.4,-86.65,VFR,"{""x"": -9645833.8772, ""y"": 3555070.433799997, ""...",Florida
6084,6003,KQHY,2024-06-14 22:55:00,Hurlbert,United States Of America,9999.0,86.4,76.6,73,220,6,0,<NA>,16000,1012.2,Few Clouds at 2900 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,97.4,30.4,-86.65,VFR,"{""x"": -9645833.8772, ""y"": 3555070.433799997, ""...",Florida
6093,1027,K2J3,2023-11-10 06:10:00,Louisville,United States Of America,97.0,57.2,56.7,98,0,0,0,<NA>,16000,1017.3,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,<NA>,<NA>,32.986,-82.3827,VFR,"{""x"": -9170800.2142, ""y"": 3893445.844999999, ""...",Georgia


In [134]:
sdfUSA.dtypes

OBJECTID                 Int64
ICAO            string[python]
OBS_DATETIME    datetime64[ns]
STATION_NAME    string[python]
COUNTRY                 object
ELEVATION              Float64
TEMP                   Float64
DEW_POINT              Float64
R_HUMIDITY               Int32
WIND_DIRECT              Int32
WIND_SPEED               Int32
WIND_GUST                Int32
WIND_CHILL             Float64
VISIBILITY               Int32
PRESSURE               Float64
SKY_CONDTN      string[python]
WEATHER         string[python]
REMARKS         string[python]
HEAT_INDEX             Float64
LATITUDE               Float64
LONGITUDE              Float64
FLT_CATEGORY    string[python]
SHAPE                 geometry
State                   object
dtype: object

So, let’s shortly describe what you will know after reading the blog post:
1.	how to split a DataFrame into a group;
2.	how select a group in the grouped DataFrame and filter the grouping data;
3.	how aggregate grouping data and how apply many function at once to the grouping data;


### Splitting of a DataFrame into groups

[[back to top]](#Table-of-Contents)

pandas DataFrames can be split on any of their axes. Grouping denotes the providence of a mapping of labels to group names. Method `groupby()` is provided in pandas for grouping. The `groupby()` function returns a `GroupBy` object, but essentially describes how the rows of the original data set has been split. 


In [135]:
sdfUSA_grouped = sdfUSA.groupby('State')
sdfUSA_grouped

In [136]:
sdfUSA_grouped['COUNTRY']

Use `list()` to show what a grouping looks like

In [137]:
list(sdfUSA_grouped)

[('Alaska',       OBJECTID  ICAO  ...                                              SHAPE   State
5950      6060  PAEC  ...  {"x": -16651503.3913, "y": 9072352.268600002, ...  Alaska
5951      3767  PAHP  ...  {"x": -18495733.3953, "y": 8746212.867399998, ...  Alaska
5952      5937  PALR  ...  {"x": -16528717.993, "y": 10299995.503399998, ...  Alaska
5953      5725  PAOH  ...  {"x": -15078002.389, "y": 7990039.264200002, "...  Alaska
5954      4180  PAWI  ...  {"x": -17812565.6803, "y": 11279281.076499999,...  Alaska

[5 rows x 24 columns]), ('California',       OBJECTID  ...       State
6065      4922  ...  California
6066       517  ...  California

[2 rows x 24 columns]), ('Florida',       OBJECTID  ...    State
6083      6002  ...  Florida
6084      6003  ...  Florida

[2 rows x 24 columns]), ('Georgia',       OBJECTID  ...    State
6093      1027  ...  Georgia
6094      1014  ...  Georgia
6095      2936  ...  Georgia

[3 rows x 24 columns]), ('Hawaii',       OBJECTID  ICAO  ...    

As it can be easily seen, the `GroupBy` object is a dictionary whose keys are the computed unique groups and corresponding values being the axis labels belonging to each group.

In [138]:
for key, value in list(sdfUSA_grouped)[:3]:
# if you want to see more values uncomment row below and comment the previous row
#for key, value in short_grouped:
    print (key)
    print (value)

Alaska
      OBJECTID  ICAO  ...                                              SHAPE   State
5950      6060  PAEC  ...  {"x": -16651503.3913, "y": 9072352.268600002, ...  Alaska
5951      3767  PAHP  ...  {"x": -18495733.3953, "y": 8746212.867399998, ...  Alaska
5952      5937  PALR  ...  {"x": -16528717.993, "y": 10299995.503399998, ...  Alaska
5953      5725  PAOH  ...  {"x": -15078002.389, "y": 7990039.264200002, "...  Alaska
5954      4180  PAWI  ...  {"x": -17812565.6803, "y": 11279281.076499999,...  Alaska

[5 rows x 24 columns]
California
      OBJECTID  ...       State
6065      4922  ...  California
6066       517  ...  California

[2 rows x 24 columns]
Florida
      OBJECTID  ...    State
6083      6002  ...  Florida
6084      6003  ...  Florida

[2 rows x 24 columns]


Besides, you may to see the first and the last item of each group

In [139]:
sdfUSA_grouped.first().head(10)

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,FLT_CATEGORY,SHAPE
State,,,,,,,,,,,,,,,,,,,,,,,
Alaska,6060,PAEC,2024-01-16 16:51:00,Chulitna,United States Of America,389.0,30.2,32.0,108,40,19,0,20.7,11000,1013.2,Overcast Cloud Deck at 1300 meters AGL,No significant weather present at this time.,* Maintenance REQUIRED - readings may be compr...,<NA>,62.886,-149.583,VFR,"{""x"": -16651503.3913, ""y"": 9072352.268600002, ..."
California,4922,KCQT,2024-05-20 14:47:00,Los Angeles,United States Of America,51.0,60.1,52.0,75,0,0,0,<NA>,16000,1016.3,Overcast Cloud Deck at 760 meters AGL,Automated observation with no human augmentati...,3-Hour Pressure Increasing: 0.1 mb,<NA>,34.024,-118.291,MVFR,"{""x"": -13168093.885400001, ""y"": 4032025.095200..."
Florida,6002,KQBL,2024-06-14 22:55:00,Hurlbert,United States Of America,9999.0,86.0,75.7,72,220,6,0,<NA>,16000,1012.5,Few Clouds at 2700 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,96.0,30.4,-86.65,VFR,"{""x"": -9645833.8772, ""y"": 3555070.433799997, ""..."
Georgia,1027,K2J3,2023-11-10 06:10:00,Louisville,United States Of America,97.0,57.2,56.7,98,0,0,0,<NA>,16000,1017.3,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,<NA>,<NA>,32.986,-82.3827,VFR,"{""x"": -9170800.2142, ""y"": 3893445.844999999, ""..."
Hawaii,5970,PHHN,2023-09-15 03:58:00,Hana,United States Of America,20.0,75.2,73.4,94,0,0,0,<NA>,14000,1015.9,Overcast Cloud Deck at 1300 meters AGL,Automated observation with no human augmentati...,<NA>,<NA>,20.8,-156.02,VFR,"{""x"": -17368066.9536, ""y"": 2368046.6396000013,..."
Illinois,5840,KNBU,2022-12-13 08:00:00,Glenview Nas,United States Of America,<NA>,48.9,35.1,59,360,20,0,43.8,16000,1023.4,Overcast Cloud Deck at 1200 meters AGL,Automated observation with no human augmentati...,<NA>,<NA>,42.08333,-87.81667,VFR,"{""x"": -9775706.6165, ""y"": 5173470.558499999, ""..."
Iowa,5965,KDMX,2023-08-24 17:00:00,Johnston/Desmoin,United States Of America,299.0,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,<NA>,<NA>,No significant weather present at this time.,<NA>,<NA>,41.73,-93.72,<NA>,"{""x"": -10432862.6771, ""y"": 5120620.167099997, ..."
Kansas,1045,K1K1,2022-12-29 06:10:00,Benton,United States Of America,411.0,46.9,29.7,51,240,6,0,45.8,16000,1002.4,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,<NA>,<NA>,37.77,-97.12,VFR,"{""x"": -10811348.9458, ""y"": 4546985.284400001, ..."
Kentucky,985,K5M9,2022-11-09 15:35:00,Marion,United States Of America,199.0,68.2,37.6,32,0,0,0,<NA>,16000,1027.1,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,<NA>,<NA>,37.33,-88.12,VFR,"{""x"": -9809473.5287, ""y"": 4485204.895800002, ""..."


In [140]:
sdfUSA_grouped.last().head(10)

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,FLT_CATEGORY,SHAPE
State,,,,,,,,,,,,,,,,,,,,,,,
Alaska,4180,PAWI,2024-04-29 23:53:00,Wainwright,United States Of America,12.0,12.9,10.0,88,90,7,0,5.1,16000,1027.1,Scattered Clouds at 60 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,<NA>,70.637,-160.013,VFR,"{""x"": -17812565.6803, ""y"": 11279281.076499999,..."
California,517,KL52,2023-04-09 16:55:00,Oceano Cnty,United States Of America,3.0,55.4,55.4,100,250,9,0,<NA>,13000,1020.3,Overcast Cloud Deck at 760 meters AGL,Automated observation with no human augmentati...,3-Hour Pressure Increasing: 0.1 mb,<NA>,35.1,-120.62,MVFR,"{""x"": -13427356.9795, ""y"": 4177479.0622000024,..."
Florida,6003,KQHY,2024-06-14 22:55:00,Hurlbert,United States Of America,9999.0,86.4,76.6,73,220,6,0,<NA>,16000,1012.2,Few Clouds at 2900 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,97.4,30.4,-86.65,VFR,"{""x"": -9645833.8772, ""y"": 3555070.433799997, ""..."
Georgia,2936,KHME,2023-02-26 13:50:00,Metter,United States Of America,60.0,60.8,35.6,39,270,11,0,<NA>,10000,1023.9,Scattered Clouds at 1100 meters AGL,No significant weather present at this time.,<NA>,<NA>,32.37,-82.07,VFR,"{""x"": -9135990.6094, ""y"": 3811977.4724999964, ..."
Hawaii,5971,PHMU,2023-09-15 04:00:00,Kamuela/Waimea,United States Of America,809.0,66.2,64.4,94,40,28,41,<NA>,14000,1017.3,"Scattered Clouds at 210 meters AGL, Broken Clo...",Automated observation with no human augmentati...,<NA>,<NA>,20.0,-155.67,VFR,"{""x"": -17329105.1318, ""y"": 2273030.927000001, ..."
Illinois,5840,KNBU,2022-12-13 08:00:00,Glenview Nas,United States Of America,<NA>,48.9,35.1,59,360,20,0,43.8,16000,1023.4,Overcast Cloud Deck at 1200 meters AGL,Automated observation with no human augmentati...,<NA>,<NA>,42.08333,-87.81667,VFR,"{""x"": -9775706.6165, ""y"": 5173470.558499999, ""..."
Iowa,5965,KDMX,2023-08-24 17:00:00,Johnston/Desmoin,United States Of America,299.0,<NA>,<NA>,<NA>,<NA>,0,0,<NA>,<NA>,<NA>,<NA>,No significant weather present at this time.,<NA>,<NA>,41.73,-93.72,<NA>,"{""x"": -10432862.6771, ""y"": 5120620.167099997, ..."
Kansas,1045,K1K1,2022-12-29 06:10:00,Benton,United States Of America,411.0,46.9,29.7,51,240,6,0,45.8,16000,1002.4,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,<NA>,<NA>,37.77,-97.12,VFR,"{""x"": -10811348.9458, ""y"": 4546985.284400001, ..."
Kentucky,985,K5M9,2022-11-09 15:35:00,Marion,United States Of America,199.0,68.2,37.6,32,0,0,0,<NA>,16000,1027.1,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,<NA>,<NA>,37.33,-88.12,VFR,"{""x"": -9809473.5287, ""y"": 4485204.895800002, ""..."


### Selection and filtering

[[back to top]](#Table-of-Contents)

Functions of descriptive statistic like `sum()`, `count()`, `max()`, `min()`, `mean()` can be quickly applied to the `GroupBy` object to obtain summary statistics for each group – an immensely useful function. The same statement is valid also for functions like `describe()` which return general information about an object. 

Let’s show a few examples of descriptive statistic function application to the `GroupBy` object

In [141]:
sdfUSA_grouped.count().head(10)

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,FLT_CATEGORY,SHAPE
State,,,,,,,,,,,,,,,,,,,,,,,
Alaska,5,5,5,5,5,4,5,5,5,5,5,5,3,5,5,5,5,2,0,5,5,5,5
California,2,2,2,2,2,2,2,2,2,2,2,2,0,2,2,1,2,1,0,2,2,1,2
Florida,2,2,2,2,2,2,2,2,2,1,2,2,0,2,2,2,2,1,2,2,2,2,2
Georgia,3,3,3,3,3,3,3,3,3,3,3,3,0,3,3,3,3,0,0,3,3,3,3
Hawaii,3,3,3,3,3,3,2,2,2,3,3,3,0,1,3,3,3,0,0,3,3,1,3
Illinois,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1
Iowa,1,1,1,1,1,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,1
Kansas,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1
Kentucky,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,0,0,1,1,1,1


In [144]:
sdfUSA_grouped.sum(numeric_only=True).add_prefix('sum_of_').head(10)  
# calculate the sum only for numeric columns, 
# all columns with other types will be ignored and missed
# we have add also common prefics to all columns. It's very convenient

,sum_of_OBJECTID,sum_of_ELEVATION,sum_of_TEMP,sum_of_DEW_POINT,sum_of_R_HUMIDITY,sum_of_WIND_DIRECT,sum_of_WIND_SPEED,sum_of_WIND_GUST,sum_of_WIND_CHILL,sum_of_VISIBILITY,sum_of_PRESSURE,sum_of_HEAT_INDEX,sum_of_LATITUDE,sum_of_LONGITUDE
State,,,,,,,,,,,,,,
Alaska,25669,988.0,198.9,180.5,440,380,120,106,89.5,94200,5043.0,0.0,320.651,-759.674
California,5439,54.0,115.5,107.4,175,250,9,0,0.0,29000,2036.6,0.0,69.124,-238.911
Florida,12005,19998.0,172.4,152.3,145,220,12,0,0.0,32000,2024.7,193.4,60.8,-173.3
Georgia,4977,364.0,178.8,126.1,173,540,18,0,0.0,42000,3051.0,0.0,98.956,-247.5827
Hawaii,17830,896.0,141.4,137.8,188,60,48,72,0.0,14000,3049.4,0.0,61.77,-468.36
Illinois,5840,0.0,48.9,35.1,59,360,20,0,43.8,16000,1023.4,0.0,42.08333,-87.81667
Iowa,5965,299.0,0.0,0.0,0,0,0,0,0.0,0,0.0,0.0,41.73,-93.72
Kansas,1045,411.0,46.9,29.7,51,240,6,0,45.8,16000,1002.4,0.0,37.77,-97.12
Kentucky,985,199.0,68.2,37.6,32,0,0,0,0.0,16000,1027.1,0.0,37.33,-88.12


In [145]:
sdfUSA_grouped.mean(numeric_only=True)

,OBJECTID,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,HEAT_INDEX,LATITUDE,LONGITUDE
State,,,,,,,,,,,,,,
Alaska,5133.8,247.0,39.78,36.1,88.0,76.0,24.0,21.2,29.833333,18840.0,1008.6,<NA>,64.1302,-151.9348
California,2719.5,27.0,57.75,53.7,87.5,125.0,4.5,0.0,<NA>,14500.0,1018.3,<NA>,34.562,-119.4555
Florida,6002.5,9999.0,86.2,76.15,72.5,220.0,6.0,0.0,<NA>,16000.0,1012.35,96.7,30.4,-86.65
Georgia,1659.0,121.333333,59.6,42.033333,57.666667,180.0,6.0,0.0,<NA>,14000.0,1017.0,<NA>,32.985333,-82.527567
Hawaii,5943.333333,298.666667,70.7,68.9,94.0,20.0,16.0,24.0,<NA>,14000.0,1016.466667,<NA>,20.59,-156.12
Illinois,5840.0,<NA>,48.9,35.1,59.0,360.0,20.0,0.0,43.8,16000.0,1023.4,<NA>,42.08333,-87.81667
Iowa,5965.0,299.0,<NA>,<NA>,<NA>,<NA>,0.0,0.0,<NA>,<NA>,<NA>,<NA>,41.73,-93.72
Kansas,1045.0,411.0,46.9,29.7,51.0,240.0,6.0,0.0,45.8,16000.0,1002.4,<NA>,37.77,-97.12
Kentucky,985.0,199.0,68.2,37.6,32.0,0.0,0.0,0.0,<NA>,16000.0,1027.1,<NA>,37.33,-88.12


In [146]:
sdfUSA.groupby(['State'])['TEMP'].mean()
# calculate sum for a single column
# it works more quickly then movies.groupby(['State']).mean()['TEMP'] becuase it sheads the data it does not need before the mean operation

State
Alaska           39.78
California       57.75
Florida           86.2
Georgia           59.6
Hawaii            70.7
Illinois          48.9
Iowa              <NA>
Kansas            46.9
Kentucky          68.2
Louisiana         8.15
Minnesota         43.0
Mississippi       <NA>
Montana           67.1
Nebraska          78.4
New Jersey        68.5
New York          42.8
Ohio              35.6
Oklahoma          71.6
Texas            50.65
Utah              57.2
West Virginia     18.9
Name: TEMP, dtype: Float64

In [148]:
sdfUSA.groupby(['State', 'STATION_NAME']).sum(numeric_only=True)
# it is possible to group by many columns
# pay your attention that the changing of the order of 
# columns for grouping in 
# its list also changes the result table

OBJECTID  ELEVATION  ...  LATITUDE  LONGITUDE
State         STATION_NAME                           ...                     
Alaska        Chandalar Lake        5937      579.0  ...      67.5    -148.48
              Chulitna              6060      389.0  ...    62.886   -149.583
              Hoonah                5725        0.0  ...    58.108   -135.448
              Hooper Bay            3767        8.0  ...     61.52    -166.15
              Wainwright            4180       12.0  ...    70.637   -160.013
California    Los Angeles           4922       51.0  ...    34.024   -118.291
              Oceano Cnty            517        3.0  ...      35.1    -120.62
Florida       Hurlbert             12005    19998.0  ...      60.8     -173.3
Georgia       Greensboro            1014      207.0  ...      33.6     -83.13
              Louisville            1027       97.0  ...    32.986   -82.3827
              Metter                2936       60.0  ...     32.37     -82.07
Hawaii        Hana                  5970       20.0  ...      20.8    -156.02
              Kamuela/Waimea        5971      809.0  ...      20.0    -155.67
              Lahaina/West Mau      5889       67.0  ...     20.97    -156.67
Illinois      Glenview Nas          5840        0.0  ...  42.08333  -87.81667
Iowa          Johnston/Desmoin      5965      299.0  ...     41.73     -93.72
Kansas        Benton                1045      411.0  ...     37.77     -97.12
Kentucky      Marion                 985      199.0  ...     37.33     -88.12
Louisiana     Boothville Hel         498        1.0  ...     29.35     -89.43
              Main Pass 289C         150       32.0  ...     29.25     -88.43
              New Orleans/Supe       107        0.0  ...     29.95     -90.07
Minnesota     Tofte (Ramos)         6104      241.0  ...     47.58     -90.83
Mississippi   West Delta 27A        5832       31.0  ...    29.121    -89.547
Montana       Forsyth               5683      831.0  ...   46.2695    -106.62
              Libby                 1618      793.0  ...   48.2834   -115.492
Nebraska      Mullen                5691        0.0  ...     42.05  -101.0667
New Jersey    Lakewood              5977       13.0  ...     40.07     -74.18
New York      Monticello            3856      427.0  ...   41.7056   -74.8004
Ohio          Dayton-Jackson         585      285.0  ...   39.6935   -83.9896
Oklahoma      Pryor                  631      190.0  ...     36.22     -95.33
Texas         Guadalupe Pass        4754     1664.0  ...   31.8327   -104.809
              Lampasas              2260      356.0  ...   31.1059   -98.1975
Utah          Panguitch              175     2060.0  ...     37.85    -112.38
West Virginia Pt Pleasant           1015      195.0  ...     38.92      -82.1

[34 rows x 14 columns]

It is possible also to show all groups

In [149]:
sdfUSA_grouped.groups

{'Alaska': [5950, 5951, 5952, 5953, 5954], 'California': [6065, 6066], 'Florida': [6083, 6084], 'Georgia': [6093, 6094, 6095], 'Hawaii': [6101, 6102, 6103], 'Illinois': [6104], 'Iowa': [6108], 'Kansas': [6115], 'Kentucky': [6116], 'Louisiana': [6120, 6121, 6122], 'Minnesota': [6125], 'Mississippi': [6126], 'Montana': [6127, 6128], 'Nebraska': [6135], 'New Jersey': [6137], 'New York': [6138], 'Ohio': [6157], 'Oklahoma': [6158], 'Texas': [6190, 6191], 'Utah': [6202], 'West Virginia': [6209]}

and display the content only of necessary group

In [150]:
sdfUSA_grouped.get_group("Mississippi")

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,FLT_CATEGORY,SHAPE,State
6126,5832,KDLP,2024-02-10 23:05:00,West Delta 27A,United States Of America,31.0,<NA>,<NA>,<NA>,180,17,31,<NA>,16000,1008.8,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,<NA>,<NA>,29.121,-89.547,VFR,"{""x"": -9968326.4421, ""y"": 3391055.656199999, ""...",Mississippi


In [152]:
sdfUSA_grouped.get_group("Mississippi").mean(numeric_only=True)

OBJECTID        5832.0
ELEVATION         31.0
TEMP              <NA>
DEW_POINT         <NA>
R_HUMIDITY        <NA>
WIND_DIRECT      180.0
WIND_SPEED        17.0
WIND_GUST         31.0
WIND_CHILL        <NA>
VISIBILITY     16000.0
PRESSURE        1008.8
HEAT_INDEX        <NA>
LATITUDE        29.121
LONGITUDE      -89.547
dtype: object

We have just one helpful and powerful option for you! It is `filter()`.

Suppose you need remain only those groups where there are less than 5 items. Sure, you may hardly calculate amount of items in each group and then remove not satisfying for condition. But it can be done very easily.

In [153]:
sdfUSA_grouped.filter(lambda x: len(x) < 5).sort_values("State")

,OBJECTID,ICAO,OBS_DATETIME,STATION_NAME,COUNTRY,ELEVATION,TEMP,DEW_POINT,R_HUMIDITY,WIND_DIRECT,WIND_SPEED,WIND_GUST,WIND_CHILL,VISIBILITY,PRESSURE,SKY_CONDTN,WEATHER,REMARKS,HEAT_INDEX,LATITUDE,LONGITUDE,FLT_CATEGORY,SHAPE,State
6065,4922,KCQT,2024-05-20 14:47:00,Los Angeles,United States Of America,51.0,60.1,52.0,75,0,0,0,<NA>,16000,1016.3,Overcast Cloud Deck at 760 meters AGL,Automated observation with no human augmentati...,3-Hour Pressure Increasing: 0.1 mb,<NA>,34.024,-118.291,MVFR,"{""x"": -13168093.885400001, ""y"": 4032025.095200...",California
6066,517,KL52,2023-04-09 16:55:00,Oceano Cnty,United States Of America,3.0,55.4,55.4,100,250,9,0,<NA>,13000,1020.3,<NA>,Automated observation with no human augmentati...,<NA>,<NA>,35.1,-120.62,<NA>,"{""x"": -13427356.9795, ""y"": 4177479.0622000024,...",California
6083,6002,KQBL,2024-06-14 22:55:00,Hurlbert,United States Of America,9999.0,86.0,75.7,72,<NA>,6,0,<NA>,16000,1012.5,Few Clouds at 2700 meters AGL,Automated observation with no human augmentati...,<NA>,96.0,30.4,-86.65,VFR,"{""x"": -9645833.8772, ""y"": 3555070.433799997, ""...",Florida
6084,6003,KQHY,2024-06-14 22:55:00,Hurlbert,United States Of America,9999.0,86.4,76.6,73,220,6,0,<NA>,16000,1012.2,Few Clouds at 2900 meters AGL,Automated observation with no human augmentati...,* Maintenance REQUIRED - readings may be compr...,97.4,30.4,-86.65,VFR,"{""x"": -9645833.8772, ""y"": 3555070.433799997, ""...",Florida
6093,1027,K2J3,2023-11-10 06:10:00,Louisville,United States Of America,97.0,57.2,56.7,98,0,0,0,<NA>,16000,1017.3,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,<NA>,<NA>,32.986,-82.3827,VFR,"{""x"": -9170800.2142, ""y"": 3893445.844999999, ""...",Georgia
6094,1014,K3J7,2023-05-03 14:30:00,Greensboro,United States Of America,207.0,60.8,33.8,36,270,7,0,<NA>,16000,1009.8,Sky Clear below 3700 meters AGL,Automated observation with no human augmentati...,<NA>,<NA>,33.6,-83.13,VFR,"{""x"": -9253989.2696, ""y"": 3975217.4293000028, ...",Georgia
6095,2936,KHME,2023-02-26 13:50:00,Metter,United States Of America,60.0,60.8,35.6,39,270,11,0,<NA>,10000,1023.9,Scattered Clouds at 1100 meters AGL,No significant weather present at this time.,<NA>,<NA>,32.37,-82.07,VFR,"{""x"": -9135990.6094, ""y"": 3811977.4724999964, ...",Georgia
6101,5970,PHHN,2023-09-15 03:58:00,Hana,United States Of America,20.0,75.2,73.4,94,0,0,0,<NA>,<NA>,1015.9,Overcast Cloud Deck at 1300 meters AGL,Automated observation with no human augmentati...,<NA>,<NA>,20.8,-156.02,<NA>,"{""x"": -17368066.9536, ""y"": 2368046.6396000013,...",Hawaii
6102,5889,PHJH,2023-01-27 04:05:00,Lahaina/West Mau,United States Of America,67.0,<NA>,<NA>,<NA>,20,20,31,<NA>,<NA>,1016.2,"Scattered Clouds at 430 meters AGL, Scattered ...","Light Drizzle, Sand Storm",<NA>,<NA>,20.97,-156.67,<NA>,"{""x"": -17440424.6226, ""y"": 2388301.7677999996,...",Hawaii
6103,5971,PHMU,2023-09-15 04:00:00,Kamuela/Waimea,United States Of America,809.0,66.2,64.4,94,40,28,41,<NA>,14000,1017.3,"Scattered Clouds at 210 meters AGL, Broken Clo...",Automated observation with no human augmentati...,<NA>,<NA>,20.0,-155.67,VFR,"{""x"": -17329105.1318, ""y"": 2273030.927000001, ...",Hawaii


One more example:

In [156]:
sdfUSA_grouped.filter(lambda x: x['TEMP'].mean(numeric_only=True) > 50 and x['R_HUMIDITY'].mean(numeric_only=True) > 50)

TypeError: boolean value of NA is ambiguous

### Aggregation

[[back to top]](#Table-of-Contents)

We have seen previously, pandas allows to apply any function to a Series or to a DataFrame. Grouping is not an exception and you may use the same method `apply()` directly with `GroupBy` objects.  Aggregation can be done via the `aggregate()` or equivalently `agg()` method

In [157]:
sdfUSA_grouped.agg(np.sum).head(10)
# or state_grouped.aggregate(np.sum)

TypeError: datetime64 type does not support sum operations

Sure, above variant is equivalent to `state_grouped.sum()`.
One of the advantages of `agg()` method is the possibility of application of many functions to one and same dataset at once

In [255]:
# at first let's write the function which returns amount of notnull values 
def count_not_null(x):
    return len([i for i in x if pd.notnull(i)])
sdfUSA_grouped[['TEMP', 'R_HUMIDITY', "WIND_SPEED"]].agg([np.sum, np.mean, count_not_null]).head(10)

TEMP             ... WIND_SPEED               
                sum       mean  ...       mean count_not_null
State                           ...                          
Alabama      2572.8  62.751220  ...   8.951220             41
Alaska       1597.3  12.778400  ...  15.867188            128
Arizona      2132.7  50.778571  ...   7.523810             42
Arkansas     2098.6  58.294444  ...  15.194444             36
California   6184.2  47.570769  ...   5.893130            131
Colorado     1656.1  25.876562  ...  13.109375             64
Connecticut   455.0  45.500000  ...  12.700000             10
Delaware      210.3  52.575000  ...  14.000000              4
Florida      6726.3  71.556383  ...   9.463158             95
Georgia      5136.1  65.013924  ...   5.936709             79

[10 rows x 9 columns]

Sure, you may apply many functions to whole DataFrame at once

Another simple aggregation example is to compute the size of each group. This is included in `GroupBy` as the `size()` method. It returns a Series whose indexes are the group names and whose values are the sizes of each group.

In [257]:
sdfUSA_grouped.size()

State
Alabama                41
Alaska                128
Arizona                42
Arkansas               36
California            131
Colorado               64
Connecticut            10
Delaware                4
Florida                95
Georgia                79
Guam                    2
Hawaii                 13
Idaho                  24
Illinois               58
Indiana                41
Iowa                   60
Johnston/Wake/Xmas      2
Kansas                 55
Kentucky               38
Louisiana              51
Maine                  17
Marshall Islands        1
Maryland               22
Massachusetts          23
Michigan               75
Minnesota             100
Mississippi            33
Missouri               43
Montana                38
Nebraska               39
Nevada                 22
New Hampshire          13
New Jersey             17
New Mexico             32
New York               46
North Carolina         69
North Dakota           36
Ohio                   56
Oklaho

>### Exercise 1

> Read in the Gulf_and_Caribbean_Marine_Ecoregions.shp file (loaded into map 1) and also included in the data package.
> Use the groupby functions you learned about to create a dataframe that contains the total area (Shape_Area column) for each of unique values in the Lat_Zone field.


In [258]:
# type your code here

